<h1>Introduction</h1>

In this project, I performed sentiment analysis using an unsupervised machine learning approach on data collected from the version control website GitHub.  I explored the distributions and relationships between projects commit comments sentiments, and project features like watchers, comments time of day and the repository programming language. Data wrangling steps are performed to prepare the data for analysis. Sentiment score calculations for each commit comment is calculated, in which it required a number of preprocessing and score estimation steps.
 <br><br>
<b>-Technologies and services used:</b><br>
  <br>1.Spark Python API (PySpark) 
  <br>To perform a scalable sentiment analysis
  <br><br>2.Google BigQuery
  <br>Access GitHub data using GHTorrent which is a “scalable, queriable, offline mirror of data offered through the Github REST API”.[4]
  <br><a href="https://bigquery.cloud.google.com/dataset/ghtorrent-bq:ght">Access Google BigQuery</a>
  <br><br>3.Databricks
  <br>Run Python and SQL notebooks(Jupyter) for the project. Databricks Is a web-based platform that offers Spark ready environments for data analytics 
  <br><a href="https://community.cloud.databricks.com">Access Databricks</a>
  <br><br>4.Python
  <br>Run the program that processes the data and calculate sentiments
  <br><br>5.SQL
  <br>Manipulate the data on BigQuery and Databricks
  <br><br>6.Python NLTK

<h1>Description of the dataset</h1>
<br> 1.Projects:<br> 
“Information about repositories. A repository is always owned by a user.The forked_from field is empty unless the project is a fork in which case it contains the id of the project the project is forked from.The deleted field means that the project has been deleted from Github.The updated_at field indicates when the last full update was done for this project”.[4]
<br><br> 
2.Commits:<br>
“Unique commits.Each commit is identified globally through its sha field. If the author or the committer has not configured his Github email address, no resolution to a user entry is possible. In that case, GHTorrent generates artificial users using the provided email in the Git commit author or committer fields. If the user then configures his Github account, GHTorrent will update the artificial user accordingly.The project_id field contains a link to the project that this commit has been first associated with. This might not be the project this commit was initially pushed to, e.g. in case the fork is processed before the parent. See project_commits. The project_id field may be null when the repository has been deleted at the time the commit is processed. This situation might happen when retrospectively processing pull requests for a repository and the repository which the pull request originates from has been deleted.”[4]
<br><br> 
3.Commit comments:<br>
“Code review comments on commits.These are comments on individual commits.”[4] 
<br><br> 
4.Watchers:<br>
“Users that have starred (was watched) a project. The created_at field is only filled in accurately for starrings for which GHTorrent has recorded a corresponding event. Otherwise, it is filled in with the latest date that the corresponding user or project has been created.”[4]

<H1 >Data wrangling (Assess data structure)</H1>

<br>
1 .<b>To determine how to access the same field in multiple tables:</b><br> I verified the structure of each table in BigQuery and reviewed GHTorrent documentation to learn more about each field. For example, The id for each record in the project table is named id in the project table, repo_id in the watcher's table and project_id in the commits table. <br>  

2 .<b>To verify the data encoding:</b><br>I read BigQuery documentation and it seems that BigQuery convert all data to UTF-8 encoding . Dates are stored in the UTC standard. <br>

3 .<b>To examine the data semantics: </b> <br>I reviewed the relational database schema in GHTorrent and queried the tables to understand relationships between entities.

<h1>Data wrangling (Explore the data)</h1>
<br>
1.<b>I verified when was the data collected </b> <br>
The GHTorrent data dump used is the latest which was updated on 2018-04-01. Making my analysis current with the publicly available data in GitHub. <br><br>

2.<b>I queried GHTorrent dataset on BigQuery to examine a number of distributions including:</b> <br>
 
-<b>How many projects are there per each programming language:</b> <br>
<pre>
SELECT language, 
       Count(*) count 
FROM   [ghtorrent-bq:ght_2018_04_01.projects] 
GROUP  BY language 
ORDER  BY count DESC 
</pre>
<br>Notice how many projects have no programming language, given that its possible they are for non programming related project. 
<br>
<br>
-<b>How many watchers are there per quarter per year:</b><br>
<pre>
SELECT Year(p.created_at)    year, 
       Quarter(p.created_at) quarter, 
       Count(*)              stars 
FROM   [ghtorrent-bq:ght_2018_04_01.projects] p 
       INNER JOIN [ghtorrent-bq:ght_2018_04_01.watchers] w 
               ON ( p.id = w.repo_id ) 
GROUP  BY year, 
          quarter 
ORDER  BY year ASC, 
          quarter ASC   
          
</pre>
The trend here is that the number of watchers has steadily increased reaching the peak (482K watches) on 2015. The number of watchers dropped to 481K and down to 351K in 2017.  
<br><br>
-<b>how many projects are there per year:</b><br>
<pre>
SELECT Count(*), 
       Year(created_at) y 
FROM   [ghtorrent-bq:ght_2018_04_01.projects] 
GROUP  BY y 
ORDER  BY y DESC 
</pre>
<br>
The number of projects have been increasing in the last years. Note that for 2018 we are only considering first quarter of the year. 
<br><br>
GHTorrent has a number of schemas in Google BigQuery (each correspond to a MySQL dump). The naming convention for these dumps is not consistent (ex, ght vs ght_20xx_0x_xx) , which led me to make a mistake and assume that the latest schema is the one named ght with no date attached to the name. After using the queries above I discovered my error and corrected my approach. <br><br>


3.<b>I grouped the tables based on meaningful fields to search for interesting patterns</b> and noticed that some tables have missing critical information. For example, while joining the Projects tables with Watchers and grouping by programming languages (a field in the project table). I noticed that the second most watched programming language is null. During my sample selection phase, I filtered out all projects with an empty/null programming language. 
Example of my queries: 
<pre>
SELECT Count(*) count, 
       p.language 
FROM   [ghtorrent-bq:ght_2018_04_01.watchers] w 
       INNER JOIN [ghtorrent-bq:ght_2018_04_01.projects] p 
               ON ( w.repo_id = p.id ) 
GROUP  BY p.language 
ORDER  BY count DESC  
</pre>

<H1>Data wrangling (Verify data quality)</H1><br>
1.<b>I verified the timestamps </b>used in many of the tables and it turned out this format is not compatible with Spark. TimeStamps in BigQuery are stored in (yyyy-mm-dd HH:mm:ss.ssss UTC) while Spark is compatible with (yyyy-mm-dd hh:mm:ss).During my sample selection on Google BigQuery, I used the SQL function (STRFTIME_UTC_USEC) to convert the timestamp to the following format '%Y-%m-%d %H:%M:%S'. Also, BigQuery stores all timestamp in the Coordinated Universal Time standard UDC and this simplify the analysis

2.<b>I also did a number of verification and correction steps to processes the comments text</b> which include: 
<ul>
<li>Checking if the comment text is empty(null). While choosing my sample dataset I filtered out all these comments</li>
<li>Determined the written language for each comment to remove non english comments. This was done using Langdetect package which is a standalone Language Identification tool accessible through https://pypi.org/project/langdetect/ </li>
<li>Remove all punctuations from comments to prevent errors. I used a Python defined function to handle this </li>
<li>All hyperlinks were removed from the comments because they don’t contribute to the sentiment analysis</li>
</ul>

<H1>Data wrangling (Select a Sample Dataset)</H1>

I selected a small sample from GHTorrent to perform the sentiment analysis on. My sample include projects in programming languages that have the most commit comments in GitHub. The sample includes 1000 project for each of the top 9 programming languages in terms of commit comments.  

<b>The following is the query that helped <b>determine the top 9 programming languages</b> in terms of commit comments:</b> 
<pre>
SELECT Count(*)   count, 
       p.language AS lan 
FROM   [ghtorrent-bq:ght_2018_04_01.projects] p 
       INNER JOIN [ghtorrent-bq:ght_2018_04_01.commits] c 
               ON ( p.id = c.project_id ) 
       INNER JOIN [ghtorrent-bq:ght_2018_04_01.commit_comments] cc 
               ON ( c.id = cc.commit_id ) 
GROUP  BY lan 
ORDER  BY count DESC 
</pre>

<b>To select a sample form the project table I used this query and I created a new table in BigQuery named -199320:yourFolderName.projects</b>: 
<pre>
SELECT id AS id, ,owner_id AS owner_id, language AS language FROM (SELECT p.id AS id, p.url AS url, p.owner_id AS owner_id, p.name AS name, p.description AS description, p.language AS language, p.created_at AS created_at, p.forked_from AS forked_from, p.deleted AS deleted, p.updated_at AS updated_at FROM [ghtorrent-bq:ght_2018_04_01.projects] p WHERE id IN (SELECT project_id FROM [ghtorrent-bq:ght_2018_04_01.commits] c INNER JOIN [ghtorrent-bq:ght_2018_04_01.commit_comments] cc ON ( c.id = cc.commit_id ) WHERE cc.body IS NOT NULL) AND language = 'JavaScript' AND p.forked_from IS NULL ORDER BY id ASC LIMIT 1000), (SELECT p.id AS id, p.url AS url, p.owner_id AS owner_id, p.name AS name, p.description AS description, p.language AS language, p.created_at AS created_at, p.forked_from AS forked_from, p.deleted AS deleted, p.updated_at AS updated_at FROM [ghtorrent-bq:ght_2018_04_01.projects] p WHERE id IN (SELECT project_id FROM [ghtorrent-bq:ght_2018_04_01.commits] c INNER JOIN [ghtorrent-bq:ght_2018_04_01.commit_comments] cc ON ( c.id = cc.commit_id ) WHERE cc.body IS NOT NULL) AND language = 'Java' AND p.forked_from IS NULL ORDER BY id ASC LIMIT 1000), (SELECT p.id AS id, p.url AS url, p.owner_id AS owner_id, p.name AS name, p.description AS description, p.language AS language, p.created_at AS created_at, p.forked_from AS forked_from, p.deleted AS deleted, p.updated_at AS updated_at FROM [ghtorrent-bq:ght_2018_04_01.projects] p WHERE id IN (SELECT project_id FROM [ghtorrent-bq:ght_2018_04_01.commits] c INNER JOIN [ghtorrent-bq:ght_2018_04_01.commit_comments] cc ON ( c.id = cc.commit_id ) WHERE cc.body IS NOT NULL) AND language = 'Python' AND p.forked_from IS NULL ORDER BY id ASC LIMIT 1000), (SELECT p.id AS id, p.url AS url, p.owner_id AS owner_id, p.name AS name, p.description AS description, p.language AS language, p.created_at AS created_at, p.forked_from AS forked_from, p.deleted AS deleted, p.updated_at AS updated_at FROM [ghtorrent-bq:ght_2018_04_01.projects] p WHERE id IN (SELECT project_id FROM [ghtorrent-bq:ght_2018_04_01.commits] c INNER JOIN [ghtorrent-bq:ght_2018_04_01.commit_comments] cc ON ( c.id = cc.commit_id ) WHERE cc.body IS NOT NULL) AND language = 'Ruby' AND p.forked_from IS NULL ORDER BY id ASC LIMIT 1000), (SELECT p.id AS id, p.url AS url, p.owner_id AS owner_id, p.name AS name, p.description AS description, p.language AS language, p.created_at AS created_at, p.forked_from AS forked_from, p.deleted AS deleted, p.updated_at AS updated_at FROM [ghtorrent-bq:ght_2018_04_01.projects] p WHERE id IN (SELECT project_id FROM [ghtorrent-bq:ght_2018_04_01.commits] c INNER JOIN [ghtorrent-bq:ght_2018_04_01.commit_comments] cc ON ( c.id = cc.commit_id ) WHERE cc.body IS NOT NULL) AND language = 'PHP' AND p.forked_from IS NULL ORDER BY id ASC LIMIT 1000), (SELECT p.id AS id, p.url AS url, p.owner_id AS owner_id, p.name AS name, p.description AS description, p.language AS language, p.created_at AS created_at, p.forked_from AS forked_from, p.deleted AS deleted, p.updated_at AS updated_at FROM [ghtorrent-bq:ght_2018_04_01.projects] p WHERE id IN (SELECT project_id FROM [ghtorrent-bq:ght_2018_04_01.commits] c INNER JOIN [ghtorrent-bq:ght_2018_04_01.commit_comments] cc ON ( c.id = cc.commit_id ) WHERE cc.body IS NOT NULL) AND language = 'HTML' AND p.forked_from IS NULL ORDER BY id ASC LIMIT 1000), (SELECT p.id AS id, p.url AS url, p.owner_id AS owner_id, p.name AS name, p.description AS description, p.language AS language, p.created_at AS created_at, p.forked_from AS forked_from, p.deleted AS deleted, p.updated_at AS updated_at FROM [ghtorrent-bq:ght_2018_04_01.projects] p WHERE id IN (SELECT project_id FROM [ghtorrent-bq:ght_2018_04_01.commits] c INNER JOIN [ghtorrent-bq:ght_2018_04_01.commit_comments] cc ON ( c.id = cc.commit_id ) WHERE cc.body IS NOT NULL) AND language = 'C++' AND p.forked_from IS NULL ORDER BY id ASC LIMIT 1000), (SELECT p.id AS id, p.url AS url, p.owner_id AS owner_id, p.name AS name, p.description AS description, p.language AS language, p.created_at AS created_at, p.forked_from AS forked_from, p.deleted AS deleted, p.updated_at AS updated_at FROM [ghtorrent-bq:ght_2018_04_01.projects] p WHERE id IN (SELECT project_id FROM [ghtorrent-bq:ght_2018_04_01.commits] c INNER JOIN [ghtorrent-bq:ght_2018_04_01.commit_comments] cc ON ( c.id = cc.commit_id ) WHERE cc.body IS NOT NULL) AND language = 'C' AND p.forked_from IS NULL ORDER BY id ASC LIMIT 1000), (SELECT p.id AS id, p.url AS url, p.owner_id AS owner_id, p.name AS name, p.description AS description, p.language AS language, p.created_at AS created_at, p.forked_from AS forked_from, p.deleted AS deleted, p.updated_at AS updated_at FROM [ghtorrent-bq:ght_2018_04_01.projects] p WHERE id IN (SELECT project_id FROM [ghtorrent-bq:ght_2018_04_01.commits] c INNER JOIN [ghtorrent-bq:ght_2018_04_01.commit_comments] cc ON ( c.id = cc.commit_id ) WHERE cc.body IS NOT NULL) AND language = 'Objective-C' AND p.forked_from IS NULL ORDER BY id ASC LIMIT 1000)
</pre>
(9,000 records returned)<br>

<b>To select the watcher I used the following query:</b>
<pre>
SELECT repo_id,
       user_id,
       STRFTIME_UTC_USEC(created_at, '%Y-%m-%d %H:%M:%S') AS created_at
FROM [ghtorrent-bq:ght_2018_04_01.watchers]
WHERE repo_id IN
    (SELECT id
     FROM [yourFolderName-199320:yourFolderName.projects])
</pre>
(7,192,843 records returned)<br>

<b>To select the commits I used the following: </b>
<pre>
SELECT id,
       author_id,
       committer_id,
       project_id,
       STRFTIME_UTC_USEC(created_at, '%Y-%m-%d %H:%M:%S') AS created_at
FROM [ghtorrent-bq:ght_2018_04_01.commits]
WHERE project_id IN
    (SELECT id
     FROM [yourFolderName-199320:yourFolderName.projects])
  AND id IN
    (SELECT c.id
     FROM [ghtorrent-bq:ght_2018_04_01.commits] c
     INNER JOIN [ghtorrent-bq:ght_2018_04_01.commit_comments] cc ON (c.id = cc.commit_id)
     WHERE cc.body IS NOT NULL)
</pre>
(97,847 records returned)<br><br>
<b>To select the commit comments I used: </b>
<pre>
SELECT
  id,
  commit_id,
  user_id,
  body,
  comment_id,
  STRFTIME_UTC_USEC(created_at, '%Y-%m-%d %H:%M:%S') AS created_at
FROM [yourFolderName-199320:yourFolderName.commit_comments]

</pre>
(254,963 records returned)

<h1>Importing the dataset</h1>

In [8]:
%sh wget -P/FileStore/tables https://storage.googleapis.com/yourFolderName/watchers.csv
#downloading the files

In [10]:
%sh wget -P/FileStore/tables https://storage.googleapis.com/yourFolderName/commits.csv

In [12]:
%sh wget -P/FileStore/tables https://storage.googleapis.com/yourFolderName/projects.csv

In [14]:
%sh wget -P/FileStore/tables https://storage.googleapis.com/yourFolderName/commit_comments_v2.csv

In [16]:
dbutils.fs.ls('file:/FileStore/tables/')

<h1>Load the data in PySpark</h1>

In [17]:
from pyspark.sql.types import  *
 
#create schema for each file, to avoid wrong conversions  
projects_schema = StructType([StructField('id', IntegerType(), False)\
                  ,StructField('owner_id', IntegerType(), False)\
                  ,StructField('language', StringType(), False)])

watchers_schema = StructType([StructField('repo_id', IntegerType(), False)\
                  ,StructField('user_id', IntegerType(), False)\
                  ,StructField('created_at', TimestampType(), False)])

commits_schema = StructType([StructField('id', IntegerType(), False)\
                 ,StructField('author_id', IntegerType(), False)\
                 ,StructField('committer_id', IntegerType(), False)\
                 ,StructField('project_id', IntegerType(), False)\
                 ,StructField('created_at', TimestampType(), False)])

commit_comments_schema = StructType([StructField('id', IntegerType(), False)\
                         ,StructField('commit_id', IntegerType(), False)\
                         ,StructField('user_id', IntegerType(), False)\
                         ,StructField('body', StringType(), False)\
                         ,StructField('comment_id', IntegerType(), False)\
                         ,StructField('created_at', TimestampType(), False)])

#enforce the schema above and create the dataframes
projects_df = spark.read.csv('file:/FileStore/tables/projects.csv', header=True, schema=projects_schema)
watchers_df = spark.read.csv('file:/FileStore/tables/watchers.csv', header=True, schema=watchers_schema)
commits_df = spark.read.csv('file:/FileStore/tables/commits.csv', header=True, schema=commits_schema)
commit_comments_df = spark.read.csv('file:/FileStore/tables/commit_comments_v2.csv', header=True, schema = commit_comments_schema)

We defined  the structure of each dataset using schemas and then created the spark dataframes for each dataset (project , watchers , commits and commit_comments)

In [19]:
projects_df.printSchema()
watchers_df.printSchema()
commits_df.printSchema()
commit_comments_df.printSchema()

In [20]:
#create the tables
projects_df.createOrReplaceTempView("projects")
commits_df.createOrReplaceTempView("commits")
watchers_df.createOrReplaceTempView("watchers")
commit_comments_df.createOrReplaceTempView("commit_comments")

We create a temporary table for each dataframe

<h1>Preprocessing and Sentiment Calculation</h1>

In [22]:
%sh pip install nltk
%sh pip install langdetect

In [23]:
#Import all the packages need to preprocess the data and calculate the sentiment
import pyspark
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
from langdetect import detect
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize, pos_tag
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from nltk.corpus import sentiwordnet as swn
import sys

We import the needed Python packages

In [25]:
#Download the needed corpus 
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('punkt')
nltk.download('stopwords')

We download the needed corpus (resources) from NLTK to perfume many functions like lemmatization, stopword removal and sentiment calculation. 

'Averaged_perceptron_tagger' is used to determine the part of speech of word in a sentence.   
'Wordnet' is used to lemmatize words and determine their lemma.  
'Sentiwordnet' help in calculating the sentiment of each word.
'Punkt' is used to perform sentence tokenization and the 'Stopwords' lists each stopwords to remove from sentences.

In [27]:
#code examples used:
#1- https://github.com/dreyco676/nlp_spark/blob/master/preproc.py
#2- https://github.com/dreyco676/nlp_spark/blob/master/nlp_with_spark.ipynb

#determine the language
def check_lang(data_str):
    try:
      predict_lang = detect(data_str)
    except:
      predict_lang ='N/A'
    return predict_lang

#remove stopwords
stopwords = stopwords.words('english')
def remove_stops(data_str):
    comment = [word for word in data_str.split(" ") if word not in stopwords]
    return ' '.join(comment)

#remove whitespace
def remove_all_sapce(astring):
  return " ".join(astring.split())


#clean the text 
def remove_features(data_str):
    # compile regex
    url_re = re.compile('https?://(www.)?\w+\.\w+(/\w+)*/?')
    punc_re = re.compile('[%s]' % re.escape(string.punctuation))
    num_re = re.compile('(\\d+)')
    alpha_num_re = re.compile("^[a-z0-9_.]+$")
    # convert to lowercase
    data_str = data_str.lower()
    # remove hyperlinks
    data_str = url_re.sub(' ', data_str)
    # remove puncuation
    data_str = punc_re.sub(' ', data_str)
    # remove numeric 'words'
    data_str = num_re.sub(' ', data_str)
    # remove non a-z 0-9 characters and words shorter than 3 characters
    list_pos = 0
    cleaned_str = ''
    for word in data_str.split():
        if list_pos == 0:
            if alpha_num_re.match(word) and len(word) > 2:
                cleaned_str = word
            else:
                cleaned_str = ' '
        else:
            if alpha_num_re.match(word) and len(word) > 2:
                cleaned_str = cleaned_str + ' ' + word
            else:
                cleaned_str += ' '
        list_pos += 1
    cleaned_str2 = remove_all_sapce(cleaned_str)
    return cleaned_str2

#check to see if a row only contains whitespace
def check_blanks(data_str):
    is_blank = str(data_str.isspace())
    return is_blank

#tokenize string
def tokenize(aString):
  a = word_tokenize(aString)
  #only return values long enough
  return a

#extract part of speech
def pos(tokenized_text):
    sent_tag_list = pos_tag(tokenized_text) 
    aList = []
    for word, tag in sent_tag_list:
        tagToUse = ''
        if tag.startswith('J'):
            tagToUse= 'a'
        elif tag.startswith('N'):
            tagToUse= 'n'
        elif tag.startswith('R'):
            tagToUse= 'r'
        elif tag.startswith('V'):
            tagToUse= 'v'
        else:
            continue
        aList.append((word, tagToUse))
    return aList

#lemmatize the commit comments  
lemmatizer = WordNetLemmatizer()
def lemmatize(array_of_word_for_a_comment):
  all_words_in_comment = []
  for word in array_of_word_for_a_comment:
    lemma = lemmatizer.lemmatize(word[0], pos=word[1])
    if not lemma:
      continue
    all_words_in_comment.append([lemma,word[1]])  
  return all_words_in_comment


#calculate the sentiment 
def cal_score(array_of_lemma_tag_for_a_comment):
    alist = [array_of_lemma_tag_for_a_comment]
    totalScore = 0
    count_words_included = 0
    for word in array_of_lemma_tag_for_a_comment:
        synset_forms = list(swn.senti_synsets(word[0], word[1]))
        if not synset_forms:
            continue
        synset = synset_forms[0] 
        totalScore = totalScore + synset.pos_score() - synset.neg_score()
        count_words_included = count_words_included +1
    final_dec = ''
    if count_words_included == 0:
        final_dec = 'N/A'
    elif totalScore == 0:
        final_dec = 'Neu'        
    elif totalScore/count_words_included < 0:
        final_dec = 'Neg'
    elif totalScore/count_words_included > 0:
        final_dec = 'Pos'
    return final_dec

 We defined a number of function including: 
<ol>
  <li><b>check_lang</b> determines  the language of text. It takes strings as input and output the language or n/a if it can't processes the text. This function is used given that in this project I will only target English comments</li>
  <li><b>remove_stops</b> removes all stopwords from a sentence. Takes as input string of words and output string without stopwords. Stopwords carry no meaning for my sentiment analysis calculations</li>
  <li><b>remove_all_sapce</b> removes extra white space from string. Takes as input string of words and output string with removed extra white space</li>
  <li><b>remove_features</b> convert string to lowercase, remove hyperlinks, remove punctuations ,remove numeric 'words' and remove non a-z 0-9 characters and words. Takes as input string of words and outputs processed string. All these steps are needed to perform lemmatization and sentiment calculation </li>
  <li><b>tokenize</b> tokenize string and return a token </li>
  <li><b>pos</b> determine the part of speech for each word in a token (to be used for lemmatization and sentiment calculations). Takes a tokenized string as input and output tokenized string with each word part of speech . For example input = ('play', 'cards'), output = (('play','v'),('cards','n'))  </li>
  <li><b>lemmatize</b> lemmatize each word in a token using wordnet. takes as input tokenize string of words and their part of speech and output the lemma of each word and its part of speech </li>
  <li><b>cal_score</b> calculate the sentiment of tokenized string (sentence). takes as input lemmatize words in a sentence with their part of speech and outputs Neu, Neg, Pos or N/A (Neutral, Negative, Positive or Not Available) depending on the sentence sentiment </li>
</ol>

In [29]:
#Register functions in Spark
check_lang_udf = udf(check_lang, StringType())
remove_stops_udf = udf(remove_stops, StringType())
remove_features_udf = udf(remove_features, StringType())
tokenize_udf = udf(tokenize, ArrayType(StringType()))
pos_udf = udf(pos,ArrayType(StructType([ StructField("word", StringType(), False), StructField("tag", StringType(), False)])))
lemmatize_udf = udf(lemmatize,ArrayType(StructType([ StructField("lemma", StringType(), False), StructField("tag", StringType(), False)])))
cal_score_udf = udf(cal_score,StringType())

We register all the functions we previously defined in spark as users defined functions UDF

In [31]:
#remove null values
data = commit_comments_df.fillna({'body':''})

We remove all records that have empty comments section as they are not needed for analysis

In [33]:
#keep only the english comments 
lang_df = data.withColumn("lang", check_lang_udf(data["body"]))
en_df = lang_df.filter(lang_df["lang"] == "en")

We create a new dataframe that has only commit comments written in English. This is a necessary  step given that in this project I'm only target English comments

In [35]:
#remove noise
rm_features_df = en_df.withColumn("feat_text", remove_features_udf(en_df["body"]))
rm_features_df.head()

We create a new dataframe that contains processed commit comments (lowercase, with no hyperlinks, punctuations, numeric 'words' and non a-z 0-9 characters and words)

In [37]:
#remove stopwords
stops_df = rm_features_df.withColumn("no_stop", remove_stops_udf(rm_features_df["feat_text"]))
stops_df.head()

We create a new dataframe that contains commit comments with no stopwords as they carry no meaning for my sentiment analysis

In [39]:
#remove/filter empty 
stops_no_null = stops_df.filter(stops_df["no_stop"] <> "")

We remove all records that have empty comments section as result of processing

In [41]:
#tokenize comments
tokenized = stops_no_null.withColumn("tokenized", tokenize_udf(stops_no_null["no_stop"]))
tokenized.head()

We create a new dataframe with tokenized  commit comments

In [43]:
#tag for part of speech
pos_df = tokenized.withColumn("pos_out", pos_udf(tokenized["tokenized"]))
pos_df.head()

 We create a new dataframe with tokenized commit comments and each word part of speech

In [45]:
#lemmatize the tokens 
lem_df = pos_df.withColumn("lem_out", lemmatize_udf(pos_df["pos_out"]))
lem_df.head()

We create a new dataframe with a tokenized and lemmatize commit comments and each word part of speech

In [47]:
#calculate the sentiment
try:
  cal_df = lem_df.withColumn("score", cal_score_udf(lem_df["lem_out"]))
except:#
  sys.setrecursionlimit(2000)
  cal_df = lem_df.withColumn("score", cal_score_udf(lem_df["lem_out"]))
cal_df.head()

We create a new dataframe with the sentiment score calculation for each commit comment

In [49]:
#creat persistent table for the commit_comments with sentiment to avoid running the code above before every analysis 
cal_df.write.saveAsTable("commit_comments_score")

<h1>Sentiment Analysis Results</h1>

In [50]:
%sql 
desc commits

In [51]:
%sql 
desc watchers

In [52]:
%sql 
desc projects

In [53]:
%sql
desc commit_comments_score

In [54]:
%sql 
/*How many project do we have per programming language in this sample?*/
select p.language , count(*)
from projects p
group by p.language

This sample has 1000 project per programming language. The purpose of this query is to display how many projects are involved per programming language in this analysis

In [56]:
%sql 
/*How many commit comments do we have per programming language ?*/
SELECT  p.language language, count(*) Number_of_commit_comments
FROM 
commit_comments_score cc inner join 
commits c on (cc.commit_id = c.id) inner join  
projects p on (c.project_id = p.id) 
group by p.language
order by p.language asc 

In [57]:
%sql 
/*For every sentiment, show the percentage of programming language?*/
SELECT p.language lan,
       cc.score AS sentiment,
       count(*)
FROM commit_comments_score cc
INNER JOIN commits c ON (cc.commit_id = c.id)
INNER JOIN projects p ON (c.project_id = p.id)
GROUP BY p.language,
         sentiment
ORDER BY lan DESC

Based on the last two queries, most commit comments in the dataset are for C++, Java and Python. Given that each programing luggage has different number of processed commit comments, for this analysis it’s more appropriate to rely on averages than counts. 


The purpose of both queries is to examine the distribution of commit comments and determine the best analysis approach (counts vs averages).  
<br><b>Sentiment labels meanings:</b></br> 
Neg = Negtive <br>
Pos = Postive <br>
N/A = unprocessed data<br>
Neu = nutral <br>

In [59]:
%sql 
/*What is the % of each sentiment per programming language?*/
select a.language as language, (a.sentiment_count/b.total_count)*100 as percentage_of_each_sentiment, a.sentiment
from (SELECT  p.language language, count(*) sentiment_count, cc.score as sentiment
FROM 
commit_comments_score cc inner join 
commits c on (cc.commit_id = c.id) inner join  
projects p on (c.project_id = p.id) 
group by p.language, cc.score
order by p.language asc , sentiment asc) a

inner join 
(SELECT  p.language language, count(*) total_count
FROM 
commit_comments_score cc inner join 
commits c on (cc.commit_id = c.id) inner join  
projects p on (c.project_id = p.id) 
group by p.language
order by p.language asc) b on (a.language = b.language)

Looking at the graphs above we see how similar the sentiments are across programming languages. C has the most negative sentiment, followed by C++. For the most positive sentiments Python and Ruby scored the highest.

The purpose of the query is to examine the percentage of commit comments sentiments  per each programming language. I want to see if comments sentiments differ in each language, and the result seems consistent (little difference).

In [61]:
%sql 
/*What is the % of each sentiment per time of the day in which comments were posted?*/
SELECT a.hr AS Hour,
       (a.count_per_sentiment / b.total_count_per_hr) * 100 AS percentage_of_sentiment_per_hour,
       a.sentiment AS sentiment
FROM
  (SELECT hour(cc.created_at) AS hr,
          cc.score AS sentiment,
          count(*) AS count_per_sentiment
   FROM commit_comments_score cc
   WHERE cc.created_at IS NOT NULL
   GROUP BY hr,
            sentiment
   ORDER BY hr ASC)a
INNER JOIN
  (SELECT hour(cc.created_at) AS hr,
          count(*) AS total_count_per_hr
   FROM commit_comments_score cc
   WHERE cc.created_at IS NOT NULL
   GROUP BY hr
   ORDER BY hr ASC)b ON (a.hr = b.hr)
ORDER BY Hour ASC

In [62]:
%sql 
/*What is the % of each sentiment per the day of the week in which comments were posted?*/
SELECT a.day_name AS day_name,
       (a.count_per_sentiment / b.total_count_per_day_name) * 100 AS percentage_of_sentiment_per_day_name,
       a.sentiment AS sentiment
FROM
  (SELECT date_format(cc.created_at,'EEEE') AS day_name,
          cc.score AS sentiment,
          count(*) AS count_per_sentiment
   FROM commit_comments_score cc
   WHERE cc.created_at IS NOT NULL
   GROUP BY day_name,
            sentiment
   ORDER BY day_name ASC)a
INNER JOIN
  (SELECT date_format(cc.created_at,'EEEE') AS day_name,
          count(*) AS total_count_per_day_name
   FROM commit_comments_score cc
   WHERE cc.created_at IS NOT NULL
   GROUP BY day_name
   ORDER BY day_name ASC)b ON (a.day_name = b.day_name)
ORDER BY day_name ASC


In [63]:
%sql 
/*What is the % of each sentiment per month in which comments were posted?*/
SELECT a.month AS month,
       (a.count_per_sentiment / b.total_count_per_month) * 100 AS percentage_of_sentiment_per_month,
       a.sentiment AS sentiment
FROM
  (SELECT month(cc.created_at) AS month,
          cc.score AS sentiment,
          count(*) AS count_per_sentiment
   FROM commit_comments_score cc
   WHERE cc.created_at IS NOT NULL
   GROUP BY month,
            sentiment
   ORDER BY month ASC)a
INNER JOIN
  (SELECT month(cc.created_at) AS month,
          count(*) AS total_count_per_month
   FROM commit_comments_score cc
   WHERE cc.created_at IS NOT NULL
   GROUP BY month
   ORDER BY month ASC)b ON (a.month = b.month)
ORDER BY month ASC

Accross all the time analysis above (last three queries) it seems that results are consistent accross all the different factors (ex, different time of day). The sentiments per each factor show little variations.

The reason behind creating these queries is to investigate wither sentiment/commenters feelings are influenced by time related factors like the time of the day, day of the week and month, and based on the results there is little variations

In [65]:
%sql 
/*check watchers distrubtion*/
select count(*) project_per_bin, floor(a.stars/100) as bin
from 
(
SELECT 
        p2.id project, COUNT(*) AS stars
    FROM
        projects p2
    INNER JOIN watchers w ON (p2.id = w.repo_id)
    GROUP BY p2.id)a
    
group by bin
order by bin asc  

From the previous query notice how most of the data occurs between bin 0 and 7 , which means most of the repositories have 0 to 700 watchers. 

The reason behind this query is to determine a proper bin size to group the watchers in the next query

In [67]:
%sql /* What is the % of sentiments in each watchers bin ?*/
SELECT q1.bin as watchers_bin,
       q1.sentiment as sentiment,
       (q1.sentiment_sum/q2.total_sentiment_per_bin)*100 as percentage_of_sentiment
   FROM
  (SELECT d.sentiment sentiment,
          floor(d.stars_per_project/50) bin,
          sum(d.sentiment_count) sentiment_sum
   FROM
     (SELECT b.project project,
             c.stars stars_per_project,
             b.count_per_sentiment sentiment_count,
             b.sentiment sentiment /*,
    b.rank rank */
      FROM
        (SELECT a.*,
                RANK() OVER (PARTITION BY a.project
                             ORDER BY a.count_per_sentiment DESC) AS rank 
         FROM
           ( SELECT p.id project,
                    COUNT(*) AS count_per_sentiment,
                    cc.score AS sentiment
            FROM commit_comments_score cc
            INNER JOIN commits c ON (cc.commit_id = c.id)
            INNER JOIN projects p ON (c.project_id = p.id)
            GROUP BY p.id,
                     cc.score
            ORDER BY p.id ASC , cc.score ASC) a ) b
      INNER JOIN
        (SELECT p2.id project,
                COUNT(*) AS stars
         FROM projects p2
         INNER JOIN watchers w ON (p2.id = w.repo_id)
         GROUP BY p2.id) c ON (b.project = c.project)
      WHERE b.rank = 1) d
   WHERE stars_per_project < 700 /*most of the data*/
   GROUP BY floor(d.stars_per_project/50),
            sentiment
   ORDER BY bin ASC)q1
INNER JOIN
  (SELECT floor(stars_per_project/ 50) AS bin,
          sum(sentiment_count) total_sentiment_per_bin
   FROM
     (SELECT b.project project,
             c.stars stars_per_project,
             b.count_per_sentiment sentiment_count,
             b.sentiment sentiment /*,
    b.rank rank */
      FROM
        (SELECT a.*,
                RANK() OVER (PARTITION BY a.project
                             ORDER BY a.count_per_sentiment DESC) AS rank 
         FROM
           ( SELECT p.id project,
                    COUNT(*) AS count_per_sentiment,
                    cc.score AS sentiment
            FROM commit_comments_score cc
            INNER JOIN commits c ON (cc.commit_id = c.id)
            INNER JOIN projects p ON (c.project_id = p.id)
            GROUP BY p.id,
                     cc.score
            ORDER BY p.id ASC , cc.score ASC) a ) b
      INNER JOIN
        (SELECT p2.id project,
                COUNT(*) AS stars
         FROM projects p2
         INNER JOIN watchers w ON (p2.id = w.repo_id)
         GROUP BY p2.id) c ON (b.project = c.project)
      WHERE b.rank = 1)
   WHERE stars_per_project < 700
   GROUP BY bin
   ORDER BY bin ASC) AS q2 ON (q1.bin = q2.bin)
   order by watchers_bin asc

In this query for each project it finds the most occurring sentiment by counting each sentiment and ranking each, then group the data based on watchers count (bin) and report the percentage of each sentiment per watchers’ bin. 
The results show that generally repositories with higher watcher (<=450) have more positive comments. 

The reason behind this query is to see how the popularity of projects relate to sentiments and determine if this is something worth investigating.

In [69]:
%sql 
SELECT DATEDIFF(c.created_at, cc.created_at) AS days,
          count(*) as count_per_date_difference
   FROM commit_comments_score cc
   INNER JOIN commits c ON (cc.commit_id = c.id)
   group by days
   order by days desc 

Most commit comments occur with in the first 15 days after the commit

The reason behind this query is to determine a proper bin size to group the days in the next query

In [71]:
%sql
/*Show the commit comments sentiments based on every 5 days since the original commit*/
SELECT a.sentiment,
       a.days,
       (a.count_per_sentiment/b.count_per_months_from_day_of_comment) * 100 AS count_per_sentiment_percentage_of_total_input_per_month
FROM
  (SELECT score sentiment,
          floor(DATEDIFF(c.created_at, cc.created_at)/5) AS days,
          count(*) count_per_sentiment
   FROM commit_comments_score cc
   INNER JOIN commits c ON (cc.commit_id = c.id)
   WHERE floor(DATEDIFF(c.created_at, cc.created_at)/5) BETWEEN -2 AND 0 /*most of the data with in 15 day after commit*/
   GROUP BY days,
            sentiment
   ORDER BY days ASC , sentiment ASC) a
INNER JOIN
  (SELECT floor(DATEDIFF(c.created_at, cc.created_at)/5) AS days,
          count(*) AS count_per_months_from_day_of_comment
   FROM commit_comments_score cc
   INNER JOIN commits c ON (cc.commit_id = c.id)
   WHERE floor(DATEDIFF(c.created_at, cc.created_at)/5) BETWEEN -2 AND 0 
   GROUP BY days
   ORDER BY days ASC)b ON (a.days = b.days)
ORDER BY days desc

From the last query, the sentiment results are consistent for every five days with the first 15 days from the commit date. 

The reason behind this query is to examine wither the sentiments/commenters feelings are different over time.

<h1>To summarize</h1>
The introduction section of this notebook, highlights the key technologies and services used in this project and provides guidelines to help readers reproduce this work. The description of the dataset section provides information about each entity involved in this project to help the reader understand the dataset.  The wrangling section covers the key tasks and questions I tried to answer to prepare my dataset and select a sample. These tasks and questions are meant to help me explore the data, determine its structure and quality, and select a smaller sample. 
Importing the dataset section demonstrate how to import the sample selected into this notebook for analysis. It also has the code used to create Spark schemas and create the temporary tables. The preprocessing and sentiment calculation section starts with importing and downloading the needed Python packages (ex, NLTK, WordNetLemmatizer and detect)  and corpus(Wordnet and Sentiwordnet) to help preprocesses the data and determine the sentiment. All the functions needed to preprocess the data and calculate the sentiment are defined in this section. Each function is registered as UDF in spark to be used in a scalable way. The functions are then used on the sample dataset to produce the desired results (determine the sentiment of each commit comment). The Sentiment Analysis Results section presents the findings from applying the project method. The sentiments are similar across programming languages. The sentiments based on time related factors such as day of the week or time of day in which the commit comments was posted in, also show little difference. The result also shows that generally, repositories with higher watcher/popularity (<=450) have more positive comments.

[1]Pandarachalil, Rafeeque & Selvaraju, Sendhilkumar & G S, Mahalakshmi. (2014). Twitter Sentiment Analysis for Large-Scale Data: An Unsupervised Approach. Cognitive Computation. 7. 254-262. https://rd.springer.com/article/10.1007/s12559-014-9310-z

[2] Dennis Hsu, Melody Moh, and Teng-Sheng Moh.(2017). Mining Frequency of Drug Side Effects over a Large Twitter Dataset Using Apache Spark. In Proceedings of the 2017 IEEE/ACM International Conference on Advances in Social Networks Analysis and Mining 2017 (ASONAM '17), Jana Diesner, Elena Ferrari, and Guandong Xu (Eds.). ACM, New York, NY, USA, 915-924. DOI: https://doi.org/10.1145/3110025.3110110 

[3]Kalliamvakou, E., Gousios, G., Blincoe, K. et al.(2016). An in-depth study of the promises and perils of mining GitHub. Empir Software Eng  21: 2035. https://doi.org/10.1007/s10664-015-9393-5

[4]Georgios Gousios.(2013). The GHTorent dataset and tool suite. In Proceedings of the 10th Working Conference on Mining Software Repositories (MSR '13). IEEE Press, Piscataway, NJ, USA, 233-236.
https://dl.acm.org/citation.cfm?id=2487085.2487132